In [1]:
import hail as hl
import pandas as pd
import numpy as np
import seaborn as sns
import plotnine as pn
import matplotlib.pyplot as plt
import os.path as osp
%run ../../nb.py
%run $TUTORIAL_DIR/files.py
gab.register_timeop_magic(get_ipython(), 'hail')
data_dir = ORGANISM_RICE_3KG_DIR
hl.init() 

Running on Apache Spark version 2.4.4
SparkUI available at http://3d498b83ee57:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/tutorial/ext/snpseek/hail-20200128-1606-0.2.30-2ae07d872f43.log


In [8]:
print(hl.spark_context()._conf.toDebugString())

spark.app.id=local-1580227561974
spark.app.name=Hail
spark.driver.extraClassPath=/opt/conda/envs/hail/lib/python3.7/site-packages/hail/hail-all-spark.jar
spark.driver.host=3d498b83ee57
spark.driver.maxResultSize=0
spark.driver.memory=92G
spark.driver.port=38321
spark.executor.extraClassPath=./hail-all-spark.jar
spark.executor.id=driver
spark.hadoop.io.compression.codecs=org.apache.hadoop.io.compress.DefaultCodec,is.hail.io.compress.BGzipCodec,is.hail.io.compress.BGzipCodecTbi,org.apache.hadoop.io.compress.GzipCodec
spark.hadoop.mapreduce.input.fileinputformat.split.minsize=0
spark.jars=file:///opt/conda/envs/hail/lib/python3.7/site-packages/hail/hail-all-spark.jar
spark.kryo.registrator=is.hail.kryo.HailKryoRegistrator
spark.kryoserializer.buffer.max=1g
spark.logConf=true
spark.master=local[*]
spark.repl.local.jars=file:///opt/conda/envs/hail/lib/python3.7/site-packages/hail/hail-all-spark.jar
spark.serializer=org.apache.spark.serializer.KryoSerializer
spark.submit.deployMode=client
sp

In [ ]:
# spark.dynamicAllocation.enabled=false

In [2]:
mt_qc0 = hl.import_plink(
    bed=osp.join(data_dir, RG0_RAW_FILE + '.bed'),
    bim=osp.join(data_dir, RG0_RAW_FILE + '.bim'),
    fam=osp.join(data_dir, RG0_RAW_FILE + '.fam'),
    skip_invalid_loci=False
)

2020-01-28 13:30:30 Hail: INFO: Found 3024 samples in fam file.
2020-01-28 13:30:30 Hail: INFO: Found 29635224 variants in bim file.


In [3]:
mt_qc0.n_partitions()

2020-01-28 13:32:34 Hail: INFO: Coerced sorted dataset


668

In [4]:
%%timeop -o rg0
mt_qc0.write(osp.join(data_dir, RG0_RAW_FILE + '.mt'), overwrite=True)

2020-01-28 13:39:22 Hail: INFO: Coerced sorted dataset


Elapsed time: 1947.1 seconds


2020-01-28 14:08:38 Hail: INFO: wrote matrix table with 29635224 rows and 3024 columns in 668 partitions to /home/eczech/data/gwas/rice-snpseek/3K_RG_29mio_biallelic_SNPs_Dataset/NB_final_snp.mt


## QC

In [2]:
mt_qc0 = hl.read_matrix_table(osp.join(data_dir, RG0_RAW_FILE + '.mt'))

In [3]:
%%timeop -o rg1

def qc_1(mt):
    mt = hl.variant_qc(mt)
    mt = mt.filter_rows(mt.variant_qc.call_rate >= .8)
    mt = hl.sample_qc(mt)
    mt = mt.filter_cols(mt.sample_qc.call_rate >= .8)
    mt = hl.variant_qc(mt)
    mt = mt.filter_rows(mt.variant_qc.call_rate >= .98)
    mt = hl.sample_qc(mt)
    mt = mt.filter_cols(mt.sample_qc.call_rate >= .98)
    return mt

mt_qc1 = qc_1(mt_qc0)
print(mt_qc1.count())

(13381703, 2900)
Elapsed time: 7517.1 seconds
